In [5]:
import (
	"fmt"
	"math/rand"
	"math"
)

In [60]:
T 				 := 1.0  // Variance
sigma 		 := 0.2  // Standard deviation
K 				 := 1.22 // Strike price
s 				 := K    // Condition
iterations := 1000000

func W(T float64) float64 {return rand.NormFloat64() * math.Sqrt(T)}
func create_stock(T float64) func() float64 {
	return func() float64 {
		return s * math.Exp(- math.Pow(sigma, 2) * (T / 2.0) + sigma * W(T))
	}
}

stocks := make([]func() float64, 4)
stocks[0] = create_stock(T)
stocks[1] = create_stock(T)
stocks[2] = create_stock(T)
stocks[3] = create_stock(T)

func price(stocks []func() float64) float64 {
	// Retrive largest stock
	var max_stock float64
	for _, stock := range stocks {
		max_stock = math.Max(max_stock, stock())
	}
	return math.Max(0, max_stock - K)
}

sum := 0.0
for i := 0; i < iterations; i++ {
	sum += price(stocks)
}
option_price := sum / float64(iterations)

fmt.Println(option_price)

0.2695122302762061


19 <nil>

In [35]:
func W(T float64) float64 {return rand.NormFloat64() * math.Sqrt(T)}
func create_stock(T float64) func() float64 {
	return func() float64 {
		return s * math.Exp(- math.Pow(sigma, 2) * (T / 2.0) + sigma * W(T))
	}
}